In [1]:
import gmaps
from config import g_key
import pandas as pd 
import requests

In [2]:
challenge_gmaps_df = pd.read_csv('data/WeatherPy_challenge.csv')
challenge_gmaps_df.dtypes

City_ID                       int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [3]:
# user prompts for input min and max temp range and whether rain/snow or not
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))                    
raining_prompt = input("Do you want it to be raining? (yes/no) ")
snowing_prompt = input("Do you want it to be snowing? (yes/no) ")

# Filter DataFrames based on input and nested decision statements, and logical expressions
if (raining_prompt == 'yes') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!=0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']==0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']==0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']==0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'yes') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']==0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!=0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
else:
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!=0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!=0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
    
preferred_challenges_df.head()

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
2,2,victoria,HK,2020-04-15 16:07:28,22.29,114.16,72.00,64,89,2.24,overcast clouds,0.0,0.0
5,5,rikitea,PF,2020-04-15 16:07:28,-23.12,-134.97,75.36,67,43,1.21,scattered clouds,0.0,0.0
26,26,akyab,MM,2020-04-15 16:07:34,20.15,92.90,80.15,81,0,5.61,clear sky,0.0,0.0
29,29,ibaiti,BR,2020-04-15 16:07:35,-23.85,-50.19,72.00,50,10,3.00,clear sky,0.0,0.0
33,33,carnarvon,AU,2020-04-15 16:07:36,-24.87,113.63,75.20,73,40,19.46,scattered clouds,0.0,0.0


In [4]:
preferred_challenges_df.count()

City_ID                     156
City                        156
Country                     154
Date                        156
Lat                         156
Lng                         156
Max Temp                    156
Humidity                    156
Cloudiness                  156
Wind Speed                  156
Current Description         156
Rain inches (last 3 hrs)    156
Snow inches (last 3 hrs)    156
dtype: int64

In [5]:
# clean up the DataFrame by dropna() funtion
clean_preferred_challenges_df = preferred_challenges_df.dropna()
clean_preferred_challenges_df.count()
# make a copy for a new DataFrame 
challenge_hotel_df = clean_preferred_challenges_df[["City", "Country","Max Temp",  "Current Description","Lat", "Lng"]].copy()
challenge_hotel_df['Hotel Name'] = ''

# search nearest hotel name by using google place API 
for index, row in challenge_hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params = {'key': g_key, 'type':'lodging', 'radius' : 5000}
    params['location'] = f"{lat},{lng}"
    try: 
        r_hotels = requests.get(base_url, params= params).json()

        challenge_hotel_df.loc[index, 'Hotel Name'] = r_hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')

# Save and upload the new DataFrame as csv file
challenge_hotel_df.to_csv(path_or_buf='data/weatherPy_vacation.csv', index_label='City_ID')

challenge_hotel_df.head(10)

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,victoria,HK,72.00,overcast clouds,22.29,114.16,Mini Hotel Central
5,rikitea,PF,75.36,scattered clouds,-23.12,-134.97,Pension Maro'i
26,akyab,MM,80.15,clear sky,20.15,92.90,Hotel Memory
29,ibaiti,BR,72.00,clear sky,-23.85,-50.19,Hotel Moraes
33,carnarvon,AU,75.20,scattered clouds,-24.87,113.63,Hospitality Carnarvon
34,puerto ayora,EC,82.40,heavy intensity rain,-0.74,-90.35,Finch Bay Galapagos Hotel
35,saint-georges,GD,86.00,few clouds,12.06,-61.75,Radisson Grenada Beach Resort
38,hilo,US,71.60,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel
39,vila velha,BR,84.20,scattered clouds,-20.33,-40.29,Hotel Vitória Palace
49,kahului,US,71.60,clear sky,20.89,-156.47,Maui Seaside Hotel


In [6]:
# create a  marker layer map with a pop-up info box by gmaps dependence
gmaps.configure(api_key = g_key)
locations = challenge_hotel_df[['Lat','Lng']]
info_box_template='''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>'''

hotel_info = [info_box_template.format(**row) for index, row in challenge_hotel_df.iterrows() ]
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))